In [1]:
import random
import pandas as pd
import numpy as np
from bat_algorithm import BatAlgorithm

global data

In [2]:
class HeartRateCalculator:
    def __init__(self, resting_hr, age):
        self.resting_hr = resting_hr
        self.age = age
        self.hr_max = 220 - 0.7 * age
        
    def calculate_average_heart_rate(self, met_value):
        avg_hr = self.resting_hr + (met_value * 3.5 * self.hr_max) / 100
        return avg_hr
    
    def calculate_max_heart_rate(self):
        return self.hr_max
    
    def calculate_heart_rate(self, met_value, time, k):
        heart_rate = self.resting_hr + (self.hr_max - self.resting_hr) * ((met_value * time / 60) ** k)
        return heart_rate


In [8]:
def calculate_hr(avgHR, vecY):
    return np.mean(avgHR @ vecY)

def calculate_er(k, hr):
    return abs(k - hr)

def calculate_ev(pi, vecY):
    return np.sum(pi @ vecY)

def try_swap(y, i, j):
    """
    Try swapping the values at indices i and j in y and return the difference
    in the function value before and after the swap.
    """
    swapped_y = y
    # Replace the values at i and j with their swapped values
    swapped_y[i], swapped_y[j] = swapped_y[j], swapped_y[i]
    # Compute the difference in the function value before and after the swap
    dif_j = calculate_hr(np.array(data["avgHR"].values), y) - calculate_hr(np.array(data["avgHR"].values), swapped_y)
    return dif_j

def do_swap(y, i, k):
    """
    Swap the values at indices i and k in y.
    """
    y[i], y[k] = y[k], y[i]

def repair(y, hr, K):
    """
    Perform local search on y to minimize the function value.
    """
    t = 0
    tl = set()
    max_k = 0
    d = len(y)
    while t < d and hr > K:
        i = random.choice([x for x in range(d) if x not in tl])
        for j in range(d):
            dif_j = try_swap(y, i, j)
            if dif_j > max_k:
                max_k = dif_j
                k = j
        if max_k > 0:
            do_swap(y, i, k)
            hr -= max_k
        tl.add(i)
        t += 1
    return y, hr

def solution_to_fitnessplan(solution):
    sorted_indices = np.argsort(-solution)
    ranks = np.empty_like(sorted_indices)
    ranks[sorted_indices] = np.arange(len(solution))
    m = 90
    n = len(ranks)
    N = n*(n+1)/2
    y = np.ceil((n - ranks)*m/N)

    return y


In [4]:
data = pd.read_csv('sample_data.csv')

In [5]:
hr_calculator = HeartRateCalculator(resting_hr=70, age=25)

data["avgHR"] = data["METs"].apply(hr_calculator.calculate_average_heart_rate)
meanHR = data["avgHR"].mean()
data["ci"] = data["avgHR"] / meanHR
data["td"] = [random.randint(2, 6) for _ in range(data["ci"].shape[0])]
data["pi"] = data["ci"] * data["avgHR"] / data["td"]

In [6]:
def fitness_function(solution):
    sorted_indices = np.argsort(-solution)
    ranks = np.empty_like(sorted_indices)
    ranks[sorted_indices] = np.arange(len(solution))
    k = 190
    beta = 0.3
    m = 90
    n = len(ranks)
    N = n*(n+1)/2
    y = np.ceil((n - ranks)*m/N)
    hr = calculate_hr(np.array(data["avgHR"].values), y)
    y, hr = repair(y, hr, k)
    hr = calculate_hr(np.array(data["avgHR"].values), y)
    er = abs(k - hr)
    ev = calculate_ev(np.array(data["pi"].values), y)

    return beta * er - ev

In [15]:
# Set up Bat Algorithm
dimension = len(data["avgHR"].values)
lower_bound = 0
upper_bound = 1
population_size = 20
max_generations = 100
ba = BatAlgorithm(fitness_function, dimension, lower_bound, upper_bound, population_size, max_generations)

# Run the optimization process
best_solution, best_fitness = ba.optimize()

# Print the results
print("Best solution: ", best_solution)
print("Best fitness: ", best_fitness)
print("Best Fitness Plan: ", solution_to_fitnessplan(best_solution))

Best solution:  [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.############-] 99% (99/100)
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1.
 0. 0. 0. 0. 1. 0. 1.]
Best fitness:  -154.10126178467453
Best Fitness Plan:  [1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 3. 1. 3. 3. 4. 4. 4.
 3. 3. 3. 1. 3. 3. 3. 3. 3. 3. 3. 1. 3. 1. 3. 1. 1. 2. 3. 3. 2. 2. 2. 2.
 2. 2. 1. 1. 2. 1. 4.]
